In [7]:
import numpy as np
import pandas as pd
import os
import json

from gensim.models import word2vec



In [8]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
model = word2vec.Word2Vec.load("movie.model")
normalized_question1 = np.load(open(DATA_IN_PATH + 'normalized_question1.npy', 'rb'),allow_pickle=True)
normalized_question2 = np.load(open(DATA_IN_PATH + 'normalized_question2.npy', 'rb'),allow_pickle=True)

In [9]:
normalized_question1[0]

'what do i use to sterlise my derma roller'

In [11]:
index2word_set = set(model.wv.index2word)

In [12]:

def get_features(words, num_features):
    global index2word_set,model    
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    num_words = 0
    # 어휘 사전 준비
    #index2word_set = set(model.wv.index2word)
    for w in words:
        if w in index2word_set:
            num_words+=1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model.wv[w])

    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    if(num_words>0) : feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [13]:
vectorized_question1 = [get_features(question,40) for question in normalized_question1]

In [14]:
vectorized_question2=[get_features(question,40) for question in normalized_question2]

In [15]:
data_configs = {}
data_configs['vocab'] = model.wv.index2word
data_configs['vocab_size'] = len(model.wv.index2word)

train_data = pd.read_csv(DATA_IN_PATH+'train.csv',encoding='utf-8')

TRAIN_Q1_DATA = 'train_q1.npy'
TRAIN_Q2_DATA = 'train_q2.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

np.save(open(DATA_IN_PATH + TRAIN_Q1_DATA, 'wb'), vectorized_question1)
np.save(open(DATA_IN_PATH + TRAIN_Q2_DATA , 'wb'), vectorized_question2)
json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))

# Test Data

In [16]:
import re

test_data = pd.read_csv(DATA_IN_PATH + 'test.csv', encoding='utf-8')
# test_data = test_data.drop(test_data.tail(1217679).index,inplace=True) # drop last n rows
valid_ids = [type(x) ==int for x in test_data.test_id] 
test_data = test_data[valid_ids].drop_duplicates()

FILTERS = "([~.,!?\"':;)(])"
change_filter = re.compile(FILTERS)

test_questions1 = [str(s) for s in test_data['question1']]
test_questions2 = [str(s) for s in test_data['question2']]

filtered_test_questions1 = list()
filtered_test_questions2 = list()

for q in test_questions1:
     filtered_test_questions1.append(re.sub(change_filter, "", q).lower())
        
for q in test_questions2:
     filtered_test_questions2.append(re.sub(change_filter, "", q).lower())

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
from nltk.tokenize import word_tokenize, sent_tokenize


normalized_test1=[]
normalized_test2=[]

for str in filtered_test_questions1:
    tokens = re.sub(r"[^a-z0-9]+", " ", str.lower())
    normalized_test1.append(tokens)
for str in filtered_test_questions2:
    tokens = re.sub(r"[^a-z0-9]+", " ", str.lower())
    normalized_test2.append(tokens)

In [ ]:
vectorized_test1 = [get_features(question,40) for question in normalized_test1]
vectorized_test2=[get_features(question,40) for question in normalized_test2]

test_id = np.array(test_data['test_id'])

TEST_Q1_DATA = 'test_q1.npy'
TEST_Q2_DATA = 'test_q2.npy'
TEST_ID_DATA = 'test_id.npy'

In [ ]:
np.save(open(DATA_IN_PATH + TEST_Q1_DATA, 'wb'), np.array(vectorized_test1))
np.save(open(DATA_IN_PATH + TEST_Q2_DATA , 'wb'), np.array(vectorized_test2))
np.save(open(DATA_IN_PATH + TEST_ID_DATA , 'wb'), test_id)

In [ ]:
vectorized_test1[0]